In [ ]:
pip install SQLAlchemy

In [ ]:
pip install pymysql

In [ ]:
!pip install cryptography

In [ ]:
import json
import pandas as pd
from sqlalchemy import create_engine, text

# 1. 이미지의 접속 정보 반영
db_host = "127.0.0.1"
db_port = "3307"          # 이미지에 있는 포트
db_user = "root"
db_pass = "goodboyseongmin12!"
db_name = "crm"           # 접속 이름(crm)을 따름 (다를 경우 수정 필요)

# SQLAlchemy 접속 URL 생성
db_url = f"mysql+pymysql://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}"
engine = create_engine(db_url)

def process_marketing_message():
    print(f"📡 DB({db_host}:{db_port})에 연결을 시도합니다...")

    # --- [Step 1] DB 조회 (수정됨: state -> stage) ---
    query = """
        SELECT stage, payload_json 
        FROM handoffs 
        WHERE stage IN ('TARGET_AUDIENCE', 'SELECTED_TEMPLATE')
    """
    
    try:
        df = pd.read_sql(query, engine)
    except Exception as e:
        print(f"❌ DB 연결 또는 조회 실패: {e}")
        return

    if df.empty:
        print("⚠️ 데이터가 없습니다.")
        return

    # --- [Step 2] 데이터 파싱 ---
    
    # 1) 타겟 데이터 (수정됨: df['state'] -> df['stage'])
    target_rows = df[df['stage'] == 'TARGET_AUDIENCE']
    if target_rows.empty:
        print("⚠️ TARGET_AUDIENCE 데이터가 없습니다.")
        return
    target_json_str = target_rows.iloc[0]['payload_json']
    target_data = json.loads(target_json_str)
    
    # 2) 템플릿 데이터 (수정됨: df['state'] -> df['stage'])
    template_rows = df[df['stage'] == 'SELECTED_TEMPLATE']
    if template_rows.empty:
        print("⚠️ SELECTED_TEMPLATE 데이터가 없습니다.")
        return
    template_json_str = template_rows.iloc[0]['payload_json']
    template_data = json.loads(template_json_str)

    # --- [Step 3] 필요한 정보 추출 ---
    user_ids = target_data.get('user_ids', [])
    template_body = template_data.get('body_with_slots', "")
    
    print(f"✅ 타겟 인원: {len(user_ids)}명")
    print(f"✅ 템플릿 확인: {template_body[:30]}...") 

    # --- [Step 4] 상품 정보 (기존 동일) ---
    matched_product = {
        "product_name": "일리윤 세라마이드 아토 로션",
        "offer": "15% 쿠폰 증정",
        "cta": "amoremall://product/P001",
        "product_detail": "온가족이 쓰는 순한 보습"
    }

    # --- [Step 5] 최종 메시지 생성 (기존 동일) ---
    final_results = []
    print("\n[메시지 생성 결과]")
    for uid in user_ids:
        slot_values = {
            "customer_name": uid,
            "product_name": matched_product['product_name'],
            "offer": matched_product['offer'],
            "cta": matched_product['cta'],
            "product_detail": matched_product['product_detail']
        }
        try:
            completed_message = template_body.format(**slot_values)
            final_results.append({"user_id": uid, "message": completed_message})
            print(f"[{uid}] {completed_message}")
        except KeyError as e:
            print(f"❌ 슬롯 에러 ({uid}): {e}")

# 실행
if __name__ == "__main__":
    process_marketing_message()

In [ ]:
import json
import pandas as pd
from sqlalchemy import create_engine, text

# 1. 접속 정보
db_host = "127.0.0.1"
db_port = "3307"
db_user = "root"
db_pass = "goodboyseongmin12!"
db_name = "crm"

db_url = f"mysql+pymysql://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}"
engine = create_engine(db_url)

def process_marketing_message():
    print(f"📡 DB({db_host}:{db_port})에 연결을 시도합니다...")

    # --- [Step 1] handoffs 테이블 조회 ---
    query = """
        SELECT stage, payload_json 
        FROM handoffs 
        WHERE stage IN ('TARGET_AUDIENCE', 'SELECTED_TEMPLATE')
    """
    
    try:
        df = pd.read_sql(query, engine)
    except Exception as e:
        print(f"❌ DB 연결 또는 조회 실패: {e}")
        return

    if df.empty:
        print("⚠️ 데이터가 없습니다.")
        return

    # --- [Step 2] 데이터 파싱 ---
    
    # 1) 타겟 데이터
    target_rows = df[df['stage'] == 'TARGET_AUDIENCE']
    if target_rows.empty:
        print("⚠️ TARGET_AUDIENCE 데이터가 없습니다.")
        return
    target_json_str = target_rows.iloc[0]['payload_json']
    target_data = json.loads(target_json_str)
    
    # 2) 템플릿 데이터
    template_rows = df[df['stage'] == 'SELECTED_TEMPLATE']
    if template_rows.empty:
        print("⚠️ SELECTED_TEMPLATE 데이터가 없습니다.")
        return
    template_json_str = template_rows.iloc[0]['payload_json']
    template_data = json.loads(template_json_str)

    # --- [Step 3] 필요한 정보 추출 ---
    user_ids = target_data.get('user_ids', [])
    template_body = template_data.get('body_with_slots', "")
    
    print(f"✅ 타겟 인원(ID): {len(user_ids)}명")

    # =================================================================
    # [Step 3.5] 실제 고객 이름 조회 (추가된 부분)
    # =================================================================
    if not user_ids:
        print("⚠️ 대상 유저 ID가 없습니다.")
        return

    # ID 리스트를 SQL의 IN 절에 넣기 위해 튜플로 변환 (예: ('u_007', 'u_008'))
    # 주의: user_ids가 하나일 경우 ('u_007',) 형태로 변환됨
    ids_tuple = tuple(user_ids)
    
    # 💡 [중요] 고객 테이블 이름이 'customers'라고 가정했습니다. 
    # 실제 테이블명(예: users, member 등)에 맞춰서 아래 SQL을 수정해주세요.
    user_query = f"""
        SELECT user_id, customer_name 
        FROM customers 
        WHERE user_id IN {ids_tuple}
    """
    
    # 튜플 문법 이슈 방지 (항목이 1개일 때 SQL 문법 오류 방지용 처리)
    if len(user_ids) == 1:
        user_query = user_query.replace(str(ids_tuple), f"('{user_ids[0]}')")

    try:
        user_df = pd.read_sql(user_query, engine)
    except Exception as e:
        print(f"❌ 고객 이름 조회 실패: {e}")
        return

    # 조회한 데이터를 딕셔너리로 변환 { 'u_007': '홍길동', ... }
    # 이렇게 하면 ID로 이름을 바로 찾을 수 있습니다.
    name_map = user_df.set_index('user_id')['customer_name'].to_dict()
    # =================================================================

    # --- [Step 4] 상품 정보 ---
    matched_product = {
        "product_name": "일리윤 세라마이드 아토 로션",
        "offer": "15% 쿠폰 증정",
        "cta": "amoremall://product/P001",
        "product_detail": "온가족이 쓰는 순한 보습"
    }

    # --- [Step 5] 최종 메시지 생성 ---
    final_results = []
    print("\n[메시지 생성 결과]")
    
    for uid in user_ids:
        # 딕셔너리에서 이름을 찾고, 없으면 ID를 그대로 사용하거나 '고객'으로 대체
        real_name = name_map.get(uid, "고객") 

        slot_values = {
            "customer_name": real_name,  # 👈 여기가 uid에서 real_name으로 변경됨
            "product_name": matched_product['product_name'],
            "offer": matched_product['offer'],
            "cta": matched_product['cta'],
            "product_detail": matched_product['product_detail']
        }
        try:
            completed_message = template_body.format(**slot_values)
            final_results.append({"user_id": uid, "message": completed_message})
            print(f"[{uid}/{real_name}] {completed_message}")
        except KeyError as e:
            print(f"❌ 슬롯 에러 ({uid}): {e}")

# 실행
if __name__ == "__main__":
    process_marketing_message()

In [ ]:
import json
import pandas as pd
from sqlalchemy import create_engine, text

# 1. DB 접속 정보
db_host = "127.0.0.1"
db_port = "3307"
db_user = "root"
db_pass = "goodboyseongmin12!"
db_name = "crm"

# ✅ [확정된 스키마 정보 반영]
user_table_name = "users"          # 테이블 이름
user_pk_column = "user_id"         # ID 컬럼 (u_007 등)
user_name_column = "customer_name" # 이름 컬럼 (실제 이름)

db_url = f"mysql+pymysql://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}"
engine = create_engine(db_url)

def process_marketing_message():
    print(f"📡 DB({db_host}:{db_port})에 연결을 시도합니다...")

    # --- [Step 1] handoffs 테이블 조회 ---
    query = """
        SELECT stage, payload_json 
        FROM handoffs 
        WHERE stage IN ('TARGET_AUDIENCE', 'SELECTED_TEMPLATE')
    """
    
    try:
        df = pd.read_sql(query, engine)
    except Exception as e:
        print(f"❌ DB 연결 또는 조회 실패: {e}")
        return

    if df.empty:
        print("⚠️ 데이터가 없습니다.")
        return

    # --- [Step 2] 데이터 파싱 ---
    
    # 1) 타겟 데이터
    target_rows = df[df['stage'] == 'TARGET_AUDIENCE']
    if target_rows.empty:
        print("⚠️ TARGET_AUDIENCE 데이터가 없습니다.")
        return
    target_json_str = target_rows.iloc[0]['payload_json']
    target_data = json.loads(target_json_str)
    
    # 2) 템플릿 데이터
    template_rows = df[df['stage'] == 'SELECTED_TEMPLATE']
    if template_rows.empty:
        print("⚠️ SELECTED_TEMPLATE 데이터가 없습니다.")
        return
    template_json_str = template_rows.iloc[0]['payload_json']
    template_data = json.loads(template_json_str)

    # --- [Step 3] ID 추출 ---
    user_ids = target_data.get('user_ids', [])
    template_body = template_data.get('body_with_slots', "")
    
    print(f"✅ 타겟 인원(ID): {len(user_ids)}명")

    # =================================================================
    # [Step 3.5] users 테이블에서 실명 조회
    # =================================================================
    if not user_ids:
        print("⚠️ 대상 유저 ID가 없습니다.")
        return

    # ID 리스트를 SQL IN 절에 넣기 위한 포맷팅
    ids_tuple = tuple(user_ids)
    if len(user_ids) == 1:
        in_clause = f"('{user_ids[0]}')"
    else:
        in_clause = str(ids_tuple)

    # users 테이블 조회 쿼리
    user_query = f"""
        SELECT {user_pk_column}, {user_name_column}
        FROM {user_table_name} 
        WHERE {user_pk_column} IN {in_clause}
    """
    
    try:
        user_df = pd.read_sql(user_query, engine)
    except Exception as e:
        print(f"❌ 유저 정보 조회 실패: {e}")
        print(f"👉 쿼리: {user_query}")
        return

    # 조회된 데이터를 딕셔너리로 변환 { 'u_001': '김철수', ... }
    name_map = user_df.set_index(user_pk_column)[user_name_column].to_dict()
    # =================================================================

    # --- [Step 4] 상품 정보 ---
    matched_product = {
        "product_name": "일리윤 세라마이드 아토 로션",
        "offer": "15% 쿠폰 증정",
        "cta": "amoremall://product/P001",
        "product_detail": "온가족이 쓰는 순한 보습"
    }

    # --- [Step 5] 최종 메시지 생성 ---
    final_results = []
    print("\n[메시지 생성 결과]")
    
    for uid in user_ids:
        # DB에서 가져온 이름 매핑 (없으면 DB에 없는 ID이므로 '고객'으로 대체)
        real_name = name_map.get(uid, "고객") 

        slot_values = {
            "customer_name": real_name,   # 👈 여기에 DB에서 가져온 이름이 들어갑니다
            "product_name": matched_product['product_name'],
            "offer": matched_product['offer'],
            "cta": matched_product['cta'],
            "product_detail": matched_product['product_detail']
        }
        try:
            completed_message = template_body.format(**slot_values)
            final_results.append({"user_id": uid, "message": completed_message})
            print(f"[{uid}/{real_name}] {completed_message}")
        except KeyError as e:
            print(f"❌ 슬롯 에러 ({uid}): {e}")

# 실행
if __name__ == "__main__":
    process_marketing_message()

In [ ]:
import json
import pandas as pd
from sqlalchemy import create_engine, text

# 1. DB 접속 정보
db_host = "127.0.0.1"
db_port = "3307"
db_user = "root"
db_pass = "goodboyseongmin12!"
db_name = "crm"

# 테이블 정보
user_table_name = "users"
user_pk_column = "user_id"
user_name_column = "customer_name"
feature_table_name = "user_features"

db_url = f"mysql+pymysql://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}"
engine = create_engine(db_url)

# 상품 카탈로그 (생략 - 위와 동일)
product_catalog = {
    "피부진정": {"product_name": "시카 카밍 젤", "offer": "진정세트 20% 할인", "cta": "amoremall://product/CICA01", "product_detail": "가장 많은 분들의 고민, 붉은기 급속 진정"},
    "영양/보습": {"product_name": "일리윤 세라마이드 로션", "offer": "대용량 1+1", "cta": "amoremall://product/ILLI01", "product_detail": "건조한 계절 필수템, 꽉 찬 보습"},
    "피부진정, 영양/보습": {"product_name": "장벽 강화 고보습 크림", "offer": "런칭 특가 30%", "cta": "amoremall://product/BARRIER99", "product_detail": "무너진 장벽 해결 솔루션"},
    "주름/탄력": {"product_name": "레티놀 탄력 앰플", "offer": "7일 체험키트 증정", "cta": "amoremall://product/ANTIAGING07", "product_detail": "가장 많은 선택, 탱탱한 피부"},
    "미백/자외선차단": {"product_name": "비타민C 잡티 세럼", "offer": "첫구매 100원 딜", "cta": "amoremall://product/VITA500", "product_detail": "환한 피부톤의 비밀"},
    "블랙헤드/모공/피지": {"product_name": "화산송이 모공 마스크", "offer": "모공 브러쉬 증정", "cta": "amoremall://product/PORE00", "product_detail": "말끔한 모공 청소"}
}
default_product = {"product_name": "데일리 베이직 수분 크림", "offer": "무료 배송 쿠폰", "cta": "amoremall://product/BASIC01", "product_detail": "누구나 부담 없는 데일리 케어"}

def process_marketing_message():
    print(f"📡 DB 연결 중...")

    # =================================================================
    # 💡 [수정됨] 최신 실행 데이터만 가져오기
    # =================================================================
    # TARGET_AUDIENCE 중 가장 최근 것 1개
    query_target = """
        SELECT payload_json 
        FROM handoffs 
        WHERE stage = 'TARGET_AUDIENCE' 
        ORDER BY created_at DESC 
        LIMIT 1
    """
    
    # SELECTED_TEMPLATE 중 가장 최근 것 1개
    query_template = """
        SELECT payload_json 
        FROM handoffs 
        WHERE stage = 'SELECTED_TEMPLATE' 
        ORDER BY created_at DESC 
        LIMIT 1
    """
    
    try:
        # 각각 따로 조회해서 가장 최신 것을 가져옵니다.
        df_target = pd.read_sql(query_target, engine)
        df_template = pd.read_sql(query_template, engine)
    except Exception as e:
        print(f"❌ DB 조회 실패: {e}")
        return

    if df_target.empty or df_template.empty:
        print("⚠️ 처리할 최신 데이터가 없습니다.")
        return

    # JSON 파싱
    target_data = json.loads(df_target.iloc[0]['payload_json'])
    template_data = json.loads(df_template.iloc[0]['payload_json'])
    
    user_ids = target_data.get('user_ids', [])
    template_body = template_data.get('body_with_slots', "")
    
    print(f"✅ 최신 타겟 데이터 로드 완료: {len(user_ids)}명 대상")
    if not user_ids: return

    # --- [Step 2] 유저 이름 & 키워드 조회 (이후 로직 동일) ---
    ids_tuple = tuple(user_ids)
    in_clause = f"('{user_ids[0]}')" if len(user_ids) == 1 else str(ids_tuple)

    user_query = f"""
        SELECT u.{user_pk_column}, u.{user_name_column}, f.keyword
        FROM {user_table_name} u
        LEFT JOIN {feature_table_name} f ON u.{user_pk_column} = f.{user_pk_column}
        WHERE u.{user_pk_column} IN {in_clause}
    """
    
    try:
        user_df = pd.read_sql(user_query, engine)
    except Exception as e:
        print(f"❌ 유저 정보 조회 실패: {e}")
        return

    # --- [Step 3] 다수결 상품 선정 ---
    valid_keywords = user_df['keyword'].dropna()

    if valid_keywords.empty:
        winning_keyword = "DEFAULT"
        selected_product = default_product
    else:
        winning_keyword = valid_keywords.value_counts().idxmax()
        count = valid_keywords.value_counts().max()
        print(f"\n🏆 [이번 실행의 대표 키워드]: {winning_keyword} (총 {count}명 해당)")
        selected_product = product_catalog.get(winning_keyword, default_product)

    print(f"👉 선정 상품: {selected_product['product_name']}\n")

    # --- [Step 4] 메시지 생성 ---
    final_results = []
    name_map = user_df.set_index(user_pk_column)[user_name_column].to_dict()

    for uid in user_ids:
        real_name = name_map.get(uid, "고객")
        
        slot_values = {
            "customer_name": real_name,
            "product_name": selected_product['product_name'],
            "offer": selected_product['offer'],
            "cta": selected_product['cta'],
            "product_detail": selected_product['product_detail']
        }

        try:
            completed_message = template_body.format(**slot_values)
            final_results.append({"user_id": uid, "message": completed_message})
            print(f"[{uid}/{real_name}] {completed_message}")
        except KeyError as e:
            print(f"❌ 슬롯 에러 ({uid}): {e}")

if __name__ == "__main__":
    process_marketing_message()

In [ ]:
import json
import pandas as pd
from sqlalchemy import create_engine, text

# 1. DB 접속 정보
db_host = "127.0.0.1"
db_port = "3307"
db_user = "root"
db_pass = "goodboyseongmin12!"
db_name = "crm"

# 테이블 정의
user_table = "users"
feature_table = "user_features"
product_table = "products"              # 상품명(product_name)이 있는 테이블
map_table = "product_concern_map"       # prod_sn과 product_concern이 매핑된 테이블

db_url = f"mysql+pymysql://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}"
engine = create_engine(db_url)

def process_marketing_message():
    print(f"📡 DB 연결 중...")

    # --- [Step 1] 최신 실행 데이터(Target) 조회 ---
    query_target = "SELECT payload_json FROM handoffs WHERE stage = 'TARGET_AUDIENCE' ORDER BY created_at DESC LIMIT 1"
    try:
        df_target = pd.read_sql(query_target, engine)
    except Exception as e:
        print(f"❌ DB 접속 실패: {e}")
        return

    if df_target.empty:
        print("⚠️ 처리할 데이터가 없습니다.")
        return

    target_data = json.loads(df_target.iloc[0]['payload_json'])
    user_ids = target_data.get('user_ids', [])
    
    if not user_ids:
        print("⚠️ 타겟 유저가 없습니다.")
        return

    # --- [Step 2] 타겟 유저들의 대표 고민(Mode) 찾기 ---
    ids_tuple = tuple(user_ids)
    in_clause = f"('{user_ids[0]}')" if len(user_ids) == 1 else str(ids_tuple)

    user_query = f"""
        SELECT f.keyword
        FROM {user_table} u
        LEFT JOIN {feature_table} f ON u.user_id = f.user_id
        WHERE u.user_id IN {in_clause}
    """
    user_df = pd.read_sql(user_query, engine)

    valid_keywords = user_df['keyword'].dropna()
    
    if valid_keywords.empty:
        print("⚠️ 유효한 피부고민 데이터가 없습니다.")
        winning_keyword = "영양/보습" # 기본값
    else:
        winning_keyword = valid_keywords.value_counts().idxmax()
        count = valid_keywords.value_counts().max()
        print(f"\n🏆 [선정된 대표 키워드]: '{winning_keyword}' (총 {count}명 해당)")

    # =================================================================
    # 💡 [핵심 로직] 복합 키워드 분리 및 상품 조회
    # =================================================================
    # 예: "피부진정, 영양/보습" -> ['피부진정', '영양/보습'] 으로 분리
    target_concerns = [k.strip() for k in winning_keyword.split(',')]
    
    print(f"🔍 매칭할 단일 키워드 목록: {target_concerns}")

    # SQL IN 절 생성을 위한 포맷팅
    if len(target_concerns) == 1:
        # 키워드가 1개일 때: WHERE product_concern = '주름/탄력'
        condition = f"= '{target_concerns[0]}'"
    else:
        # 키워드가 여러 개일 때: WHERE product_concern IN ('피부진정', '영양/보습')
        condition = f"IN {tuple(target_concerns)}"

    # JOIN 쿼리: map 테이블과 products 테이블을 prod_sn으로 연결
    product_query = f"""
        SELECT p.product_name, p.prod_sn, m.product_concern
        FROM {product_table} p
        JOIN {map_table} m ON p.prod_sn = m.prod_sn
        WHERE m.product_concern {condition}
    """

    try:
        product_df = pd.read_sql(product_query, engine)
    except Exception as e:
        print(f"❌ 상품 조회 쿼리 오류: {e}")
        print(f"👉 쿼리: {product_query}")
        return

    if product_df.empty:
        print(f"⚠️ '{winning_keyword}'에 해당하는 상품이 DB에 없습니다.")
    else:
        print(f"✅ 총 {len(product_df)}개의 상품이 매칭되었습니다.\n")
        
        # 결과 출력 (상위 10개만)
        print("[매칭된 상품 리스트 (예시)]")
        print("-" * 50)
        print(f"{'상품ID':<10} | {'카테고리':<10} | {'상품명'}")
        print("-" * 50)
        
        for idx, row in product_df.head(10).iterrows():
            print(f"{row['prod_sn']:<10} | {row['product_concern']:<10} | {row['product_name']}")
        print("-" * 50)

        # 실제 메시지 발송용 상품 1개 선정 (랜덤 혹은 첫 번째)
        selected_product_name = product_df.iloc[0]['product_name']
        print(f"\n👉 최종 추천 상품: {selected_product_name}")

if __name__ == "__main__":
    process_marketing_message()

In [ ]:
import json
import pandas as pd
from sqlalchemy import create_engine, text

# 1. DB 접속 정보
db_host = "127.0.0.1"
db_port = "3307"
db_user = "root"
db_pass = "goodboyseongmin12!"
db_name = "crm"

# 테이블 정의
user_table = "users"
feature_table = "user_features"
product_table = "products"
map_table = "product_concern_map"

db_url = f"mysql+pymysql://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}"
engine = create_engine(db_url)

def process_marketing_message():
    print(f"📡 DB 연결 중...")

    # --- [Step 1] 최신 타겟/템플릿 데이터 조회 ---
    query_target = "SELECT payload_json FROM handoffs WHERE stage = 'TARGET_AUDIENCE' ORDER BY created_at DESC LIMIT 1"
    query_template = "SELECT payload_json FROM handoffs WHERE stage = 'SELECTED_TEMPLATE' ORDER BY created_at DESC LIMIT 1"

    try:
        df_target = pd.read_sql(query_target, engine)
        df_template = pd.read_sql(query_template, engine)
    except Exception as e:
        print(f"❌ DB 접속 실패: {e}")
        return

    if df_target.empty or df_template.empty:
        print("⚠️ 처리할 데이터가 없습니다.")
        return

    target_data = json.loads(df_target.iloc[0]['payload_json'])
    template_data = json.loads(df_template.iloc[0]['payload_json'])
    
    user_ids = target_data.get('user_ids', [])
    template_body = template_data.get('body_with_slots', "")
    
    if not user_ids: return

    # --- [Step 2] 유저들의 피부고민 조회 ---
    ids_tuple = tuple(user_ids)
    in_clause = f"('{user_ids[0]}')" if len(user_ids) == 1 else str(ids_tuple)

    user_query = f"""
        SELECT u.user_id, u.customer_name, f.keyword
        FROM {user_table} u
        LEFT JOIN {feature_table} f ON u.user_id = f.user_id
        WHERE u.user_id IN {in_clause}
    """
    try:
        user_df = pd.read_sql(user_query, engine)
    except Exception as e:
        print(f"❌ 유저 정보 조회 실패: {e}")
        return

    # --- [Step 3] 최빈값(Mode) 1개 선정 ---
    valid_keywords = user_df['keyword'].dropna()
    
    if valid_keywords.empty:
        print("⛔ [중단] 유효한 피부고민 데이터가 없습니다.")
        return 

    # 최빈값 추출 (예: "피부진정, 영양/보습")
    most_frequent_raw = valid_keywords.value_counts().idxmax()
    count = valid_keywords.value_counts().max()

    # 콤마(,) 분리 후 첫 번째 키워드 선택 (예: "피부진정")
    final_single_keyword = most_frequent_raw.split(',')[0].strip()

    print(f"\n🏆 [최빈값 원본]: '{most_frequent_raw}' (총 {count}명)")
    print(f"🎯 [최종 선정된 단일 키워드]: '{final_single_keyword}'")

    # --- [Step 4] DB 조회 (상품명만 가져옴) ---
    # offer, cta, product_detail 컬럼은 쿼리에서 제외했습니다.
    product_query = f"""
        SELECT 
            p.product_name, 
            m.product_concern
        FROM {product_table} p
        JOIN {map_table} m ON p.prod_sn = m.prod_sn
        WHERE m.product_concern = '{final_single_keyword}'
    """

    try:
        product_df = pd.read_sql(product_query, engine)
    except Exception as e:
        print(f"❌ 상품 조회 실패: {e}")
        return

    if product_df.empty:
        print(f"⛔ [중단] 키워드 '{final_single_keyword}'에 해당하는 상품이 없습니다.")
        return

    # --- [Step 5] 메시지 생성 ---
    selected_product = product_df.iloc[0] # 첫 번째 상품 선택

    print(f"👉 [최종 매칭 상품]: {selected_product['product_name']}")
    print("   (CTA, Offer 등은 현재 공란으로 처리됩니다)")
    print("-" * 50)

    final_results = []
    name_map = user_df.set_index('user_id')['customer_name'].to_dict()

    print("\n[메시지 발송 미리보기]")
    for uid in user_ids:
        real_name = name_map.get(uid, "고객")
        
        # 💡 [핵심] 없는 데이터는 빈 문자열("")로 채워 에러 방지
        slot_values = {
            "customer_name": real_name,
            "product_name": selected_product['product_name'],
            "offer": "",           # 나중에 채워넣을 예정
            "cta": "",             # 나중에 채워넣을 예정
            "product_detail": ""   # 나중에 채워넣을 예정
        }

        try:
            completed_message = template_body.format(**slot_values)
            final_results.append({"user_id": uid, "message": completed_message})
            print(f"[{uid}/{real_name}] {completed_message}")
        except KeyError as e:
            print(f"❌ 슬롯 에러 ({uid}): {e}")

if __name__ == "__main__":
    process_marketing_message()

In [ ]:
pip install sentence-transformers

In [ ]:
import json
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from sklearn.metrics.pairwise import cosine_similarity
# 💡 [핵심 변경] TF-IDF 대신 임베딩 모델 사용
from sentence_transformers import SentenceTransformer

# 1. DB 접속 정보
db_host = "127.0.0.1"
db_port = "3307"
db_user = "root"
db_pass = "goodboyseongmin12!"
db_name = "crm"

# 테이블 정보
user_table = "users"
feature_table = "user_features"
product_table = "products"
map_table = "product_concern_map"
ocr_table = "product_ocr_text"

db_url = f"mysql+pymysql://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}"
engine = create_engine(db_url)

# 💡 [모델 로드] 한국어 문장 유사도 성능이 뛰어난 모델입니다. (최초 실행 시 다운로드 시간 소요됨)
# 모델은 전역 변수로 한 번만 로드하는 것이 좋습니다.
print("⏳ AI 모델 로딩 중... (잠시만 기다려 주세요)")
embedding_model = SentenceTransformer('jhgan/ko-sroberta-multitask')
print("✅ 모델 로딩 완료!")

def process_marketing_message():
    print(f"📡 DB 연결 중...")

    # --- [Step 1] 데이터 조회 ---
    query_target = "SELECT payload_json FROM handoffs WHERE stage = 'TARGET_AUDIENCE' ORDER BY created_at DESC LIMIT 1"
    query_template = "SELECT payload_json FROM handoffs WHERE stage = 'SELECTED_TEMPLATE' ORDER BY created_at DESC LIMIT 1"

    try:
        df_target = pd.read_sql(query_target, engine)
        df_template = pd.read_sql(query_template, engine)
    except Exception as e:
        print(f"❌ DB 접속 실패: {e}")
        return

    if df_target.empty or df_template.empty:
        print("⚠️ 데이터 없음.")
        return

    target_data = json.loads(df_target.iloc[0]['payload_json'])
    template_data = json.loads(df_template.iloc[0]['payload_json'])
    user_ids = target_data.get('user_ids', [])
    template_body = template_data.get('body_with_slots', "")
    
    if not user_ids: return

    # --- [Step 2] 캠페인 키워드 ---
    try:
        campaign_keywords_list = template_data['notes']['campaign_text_normalized']['keywords']
        campaign_text = " ".join(campaign_keywords_list)
        print(f"\n🎯 [캠페인 키워드]: {campaign_text}")
    except KeyError:
        campaign_text = "추천 상품"

    # --- [Step 3] 1차 필터링 ---
    ids_tuple = tuple(user_ids)
    in_clause = f"('{user_ids[0]}')" if len(user_ids) == 1 else str(ids_tuple)

    user_query = f"""
        SELECT f.keyword FROM {user_table} u
        LEFT JOIN {feature_table} f ON u.user_id = f.user_id
        WHERE u.user_id IN {in_clause}
    """
    user_df = pd.read_sql(user_query, engine)
    valid_keywords = user_df['keyword'].dropna()
    
    if valid_keywords.empty:
        print("⛔ 유저 데이터 없음.")
        return 
    
    winning_category = valid_keywords.value_counts().idxmax().split(',')[0].strip()
    print(f"🏆 [1차 필터] 카테고리: '{winning_category}'")

    # --- [Step 4] 상품 조회 ---
    product_query = f"""
        SELECT 
            p.prod_sn,
            p.product_name,
            o.keyword as db_product_keywords
        FROM {product_table} p
        JOIN {map_table} m ON p.prod_sn = m.prod_sn
        LEFT JOIN {ocr_table} o ON p.prod_sn = o.prod_sn
        WHERE m.product_concern = '{winning_category}'
    """
    try:
        candidate_df = pd.read_sql(product_query, engine)
    except Exception as e:
        print(f"❌ 상품 조회 실패: {e}")
        return

    if candidate_df.empty:
        print(f"⛔ 후보 상품 없음.")
        return

    # 전처리
    candidate_df['db_product_keywords'] = candidate_df['db_product_keywords'].fillna("").astype(str)
    candidate_df['offer'] = ""
    candidate_df['cta'] = ""
    candidate_df['product_detail'] = ""

    print(f"✅ 후보 상품 수: {len(candidate_df)}개")

    # =================================================================
    # 💡 [Step 5] 임베딩(Embedding) 기반 유사도 분석
    # =================================================================
    # 1. 텍스트 리스트 준비
    campaign_list = [campaign_text]
    product_keywords_list = candidate_df['db_product_keywords'].tolist()

    # 2. 임베딩 벡터 생성 (이 과정이 '의미'를 숫자로 바꿉니다)
    #    encode() 함수가 텍스트를 768차원 등의 벡터로 변환해줍니다.
    campaign_embedding = embedding_model.encode(campaign_list)
    product_embeddings = embedding_model.encode(product_keywords_list)

    # 3. 코사인 유사도 계산
    #    (1개 캠페인 벡터 vs N개 상품 벡터들)
    similarity_scores = cosine_similarity(campaign_embedding, product_embeddings).flatten()

    # 4. Best 상품 선정
    best_match_idx = similarity_scores.argmax()
    final_product = candidate_df.iloc[best_match_idx]

    print("\n🔍 [AI 의미 기반(Embedding) 유사도 점수]")
    print("-" * 50)
    # 상위 5개 출력
    sorted_indices = similarity_scores.argsort()[::-1][:5]
    for idx in sorted_indices:
        p_name = candidate_df.iloc[idx]['product_name']
        score = similarity_scores[idx]
        is_winner = "👈 WIN" if idx == best_match_idx else ""
        print(f" • [{p_name}] -> {score:.4f} {is_winner}")
    print("-" * 50)

    print(f"👉 [최종 선정 상품]: {final_product['product_name']}")

    # --- [Step 6] 메시지 생성 (기존 동일) ---
    final_results = []
    user_name_df = pd.read_sql(f"SELECT user_id, customer_name FROM {user_table} WHERE user_id IN {in_clause}", engine)
    name_map = user_name_df.set_index('user_id')['customer_name'].to_dict()

    print("\n[메시지 발송 미리보기]")
    for uid in user_ids:
        real_name = name_map.get(uid, "고객")
        slot_values = {
            "customer_name": real_name,
            "product_name": final_product['product_name'],
            "offer": final_product['offer'],
            "cta": final_product['cta'],
            "product_detail": final_product['product_detail']
        }
        try:
            completed_message = template_body.format(**slot_values)
            final_results.append({"user_id": uid, "message": completed_message})
            print(f"[{uid}/{real_name}] {completed_message}")
        except KeyError:
            pass

if __name__ == "__main__":
    process_marketing_message()

In [1]:
import json
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# 1. DB 접속 정보
db_host = "127.0.0.1"
db_port = "3307"
db_user = "root"
db_pass = "goodboyseongmin12!"
db_name = "crm"

# 테이블 정보
user_table = "users"
feature_table = "user_features"
product_table = "products"
map_table = "product_concern_map"
ocr_table = "product_ocr_text"

db_url = f"mysql+pymysql://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}"
engine = create_engine(db_url)

# 모델 로드 (최초 1회 실행)
print("⏳ AI 모델 로딩 중...")
embedding_model = SentenceTransformer('jhgan/ko-sroberta-multitask')
print("✅ 모델 로딩 완료!")

def process_marketing_message():
    print(f"📡 DB 연결 중...")

    # --- [Step 1] 타겟 & 템플릿 데이터 조회 ---
    query_target = "SELECT payload_json FROM handoffs WHERE stage = 'TARGET_AUDIENCE' ORDER BY created_at DESC LIMIT 1"
    query_template = "SELECT payload_json FROM handoffs WHERE stage = 'SELECTED_TEMPLATE' ORDER BY created_at DESC LIMIT 1"

    try:
        df_target = pd.read_sql(query_target, engine)
        df_template = pd.read_sql(query_template, engine)
    except Exception as e:
        print(f"❌ DB 접속 실패: {e}")
        return

    if df_target.empty or df_template.empty:
        print("⚠️ 데이터 없음.")
        return

    target_data = json.loads(df_target.iloc[0]['payload_json'])
    template_data = json.loads(df_template.iloc[0]['payload_json'])
    user_ids = target_data.get('user_ids', [])
    template_body = template_data.get('body_with_slots', "")
    
    if not user_ids: return

    # --- [Step 2] 캠페인 키워드 ---
    try:
        campaign_keywords_list = template_data['notes']['campaign_text_normalized']['keywords']
        campaign_text = " ".join(campaign_keywords_list)
        print(f"\n🎯 [캠페인 키워드]: {campaign_text}")
    except KeyError:
        campaign_text = "추천 상품"

    # --- [Step 3] 1차 필터링 (카테고리 선정) ---
    ids_tuple = tuple(user_ids)
    in_clause = f"('{user_ids[0]}')" if len(user_ids) == 1 else str(ids_tuple)

    user_query = f"""
        SELECT f.keyword FROM {user_table} u
        LEFT JOIN {feature_table} f ON u.user_id = f.user_id
        WHERE u.user_id IN {in_clause}
    """
    user_df = pd.read_sql(user_query, engine)
    valid_keywords = user_df['keyword'].dropna()
    
    if valid_keywords.empty:
        print("⛔ 유저 데이터 없음.")
        return 
    
    winning_category = valid_keywords.value_counts().idxmax().split(',')[0].strip()
    print(f"🏆 [1차 필터] 카테고리: '{winning_category}'")

    # =================================================================
    # 💡 [Step 4] 상품 및 상세 정보 조회 (수정됨)
    # =================================================================
    # p.detail_url  -> CTA 링크
    # o.detail_slot -> 상품 설명 멘트
    product_query = f"""
        SELECT 
            p.prod_sn,
            p.product_name,
            p.detail_url,                  -- 👈 CTA용 URL
            o.keyword as db_product_keywords,
            o.detail_slot                  -- 👈 멘트용 상세 설명
        FROM {product_table} p
        JOIN {map_table} m ON p.prod_sn = m.prod_sn
        LEFT JOIN {ocr_table} o ON p.prod_sn = o.prod_sn
        WHERE m.product_concern = '{winning_category}'
    """
    try:
        candidate_df = pd.read_sql(product_query, engine)
    except Exception as e:
        print(f"❌ 상품 조회 실패: {e}")
        return

    if candidate_df.empty:
        print(f"⛔ 후보 상품 없음.")
        return

    # 전처리 (Null 방지)
    candidate_df['db_product_keywords'] = candidate_df['db_product_keywords'].fillna("").astype(str)
    candidate_df['detail_url'] = candidate_df['detail_url'].fillna("")  # URL 없을 경우 대비
    candidate_df['detail_slot'] = candidate_df['detail_slot'].fillna("") # 설명 없을 경우 대비
    
    # offer는 아직 소스가 없으므로 빈 값 유지
    candidate_df['offer'] = "" 

    print(f"✅ 후보 상품 수: {len(candidate_df)}개")

    # --- [Step 5] 임베딩 유사도 분석 ---
    campaign_list = [campaign_text]
    product_keywords_list = candidate_df['db_product_keywords'].tolist()

    # 벡터화 및 비교
    campaign_embedding = embedding_model.encode(campaign_list)
    product_embeddings = embedding_model.encode(product_keywords_list)
    similarity_scores = cosine_similarity(campaign_embedding, product_embeddings).flatten()

    # Best 상품 선정
    best_match_idx = similarity_scores.argmax()
    final_product = candidate_df.iloc[best_match_idx]

    print("\n🔍 [AI 의미 기반 상품 선정 결과]")
    print("-" * 50)
    sorted_indices = similarity_scores.argsort()[::-1][:3] # 상위 3개만 로그 출력
    for idx in sorted_indices:
        p_name = candidate_df.iloc[idx]['product_name']
        score = similarity_scores[idx]
        is_winner = "👈 WIN" if idx == best_match_idx else ""
        print(f" • [{p_name}] -> {score:.4f} {is_winner}")
    print("-" * 50)

    print(f"👉 [최종 선정 상품]: {final_product['product_name']}")
    print(f"   ㄴ 상세멘트(detail_slot): {final_product['detail_slot']}")
    print(f"   ㄴ 링크(detail_url): {final_product['detail_url']}")

    # --- [Step 6] 메시지 생성 ---
    final_results = []
    user_name_df = pd.read_sql(f"SELECT user_id, customer_name FROM {user_table} WHERE user_id IN {in_clause}", engine)
    name_map = user_name_df.set_index('user_id')['customer_name'].to_dict()

    print("\n[메시지 발송 미리보기]")
    for uid in user_ids:
        real_name = name_map.get(uid, "고객")
        
        # 💡 [수정됨] 실제 DB 데이터 매핑
        slot_values = {
            "customer_name": real_name,
            "product_name": final_product['product_name'],
            "offer": final_product['offer'],          # 아직 비어있음
            "cta": final_product['detail_url'],       # 👈 products 테이블의 detail_url
            "product_detail": final_product['detail_slot'] # 👈 product_ocr_text 테이블의 detail_slot
        }

        try:
            completed_message = template_body.format(**slot_values)
            final_results.append({"user_id": uid, "message": completed_message})
            print(f"[{uid}/{real_name}] {completed_message}")
        except KeyError as e:
            print(f"❌ 슬롯 에러: {e}")

if __name__ == "__main__":
    process_marketing_message()

c:\Users\saure\miniconda3\envs\pystudy_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


⏳ AI 모델 로딩 중...
✅ 모델 로딩 완료!
📡 DB 연결 중...

🎯 [캠페인 키워드]: 
🏆 [1차 필터] 카테고리: '피부진정'
✅ 후보 상품 수: 31개

🔍 [AI 의미 기반 상품 선정 결과]
--------------------------------------------------
 • [그린티 세라마이드 크림 80ml (튜브)] -> 1.0000 👈 WIN
 • [그린티 아미노 수분 클렌징 폼 150g] -> 0.3969 
 • [그린티 세라마이드 크림 80ml (튜브)] -> 0.3559 
--------------------------------------------------
👉 [최종 선정 상품]: 그린티 세라마이드 크림 80ml (튜브)
   ㄴ 상세멘트(detail_slot): 
   ㄴ 링크(detail_url): https://www.amoremall.com/kr/ko/product/detail?onlineProdSn=66914

[메시지 발송 미리보기]
[u_007/User007] User007님, 장바구니에 있는 그린티 세라마이드 크림 80ml (튜브)를 잊지 마세요!  혜택이 준비되어 있어요. 지금 앱에서 확인해 보세요: https://www.amoremall.com/kr/ko/product/detail?onlineProdSn=66914
· 
[u_017/User017] User017님, 장바구니에 있는 그린티 세라마이드 크림 80ml (튜브)를 잊지 마세요!  혜택이 준비되어 있어요. 지금 앱에서 확인해 보세요: https://www.amoremall.com/kr/ko/product/detail?onlineProdSn=66914
· 
[u_027/User027] User027님, 장바구니에 있는 그린티 세라마이드 크림 80ml (튜브)를 잊지 마세요!  혜택이 준비되어 있어요. 지금 앱에서 확인해 보세요: https://www.amoremall.com/kr/ko/product/detail?onlineProdSn=6691